# Clearwater Modules Architecture

**Author:** Xavier Nogueira

# Installation and Setup

## Install

Carefully follow our **[Installation Instructions](README.md#getting-started)**, especially including:
- Creating a virtual environment for this repository (step 3)

## Import Python Dependancies

In [1]:
import clearwater_modules as cwm
import clearwater_modules.sorter as sorter
import numba
import random
import hvplot.xarray
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Confirm that sub-modules are imported
dir(cwm)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'base',
 'nsm1',
 'shared',
 'sorter',
 'tsm',
 'utils']

### If you get `ModuleNotFoundError`:

If you get this error:
```python
ModuleNotFoundError: No module named 'clearwater_modules'
```
Then:
1. Run the following terminal command with your local absolute path to this repo.
    - NOTE: Here we use Jupyter `!` magic command to run from the terminal via this notebook. 
2. Restart the kernel.
3. Rerun the import statements above.

See [4. Add your `ClearWater-modules-python` Path to Miniconda/Anaconda sites-packages](..ReadMe.md#4-add-your-clearwater-modules-python-path-to-minicondaanaconda-sites-packages).

# Writing/using a simple `Model` sub-class example

In this example we will be writing a `base.Model` sub-class that calculates the annual carbon sequestration in a forest for a given year timestep.

**Note:** Do not take the calculation too literally! I got it off ChatGPT in order to find a good, simple example for the code.

## Start by inheriting `base.Model` -> `CarbonSequestration(cwm.base.Model)`

In [1]:
import clearwater_modules as cwm
import clearwater_modules.sorter as sorter
import numba
import random
import hvplot.xarray
import warnings
warnings.filterwarnings("ignore")

initial_state_values = {'water_temp_c': 20, 'surface_area': 1, 'volume': 1}
meteo_parameters = {
    'air_temp_c': 20,
    'q_solar': 400,
    'sed_temp_c': 10,
    'eair_mb': 2,
    'pressure_mb': 1035,
    'cloudiness': .1,
    'wind_speed': 3,
    'wind_a': 1,
    'wind_b': 1,
    'wind_c': 1,
    'wind_kh_kw': 1
}

temp_parameters = {
    'use_sed_temp': False,
    'stefan_boltzmann': 10,
    'cp_air': 1,
    'emissivity_water': 1,
    'gravity': 9.82,
    'a0': 3333,
    'a1': 3333,
    'a2': 3333,
    'a3': 3333,
    'a4': 3333,
    'a5': 3333,
    'a6': 3333,
    'pb': 3333,
    'cps': 3333,
    'h2': 3333,
    'alphas': 3333,
    'richardson_option': True
}

tsm_model = cwm.tsm.EnergyBudget(
    initial_state_values=initial_state_values,  # mandatory
    temp_parameters=temp_parameters,
    meteo_parameters=meteo_parameters,
    track_dynamic_variables=True,  # default is true
    hotstart_dataset=None,  # default is None
    time_dim='year',  # default is "timestep"
)

#print(tsm_model.get_state_variables())
tsm_model.dataset

Initializing from dicts...
Model initialized from input dicts successfully!.


<xarray.Dataset>
Dimensions:            (year: 1, x: 1, y: 1)
Coordinates:
  * year               (year) int32 0
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/31)
    water_temp_c       (year, x, y) int32 20
    surface_area       (year, x, y) int32 1
    volume             (year, x, y) int32 1
    use_sed_temp       (x, y) bool True
    stefan_boltzmann   (x, y) int32 10
    cp_air             (x, y) int32 1
    ...                 ...
    cloudiness         (x, y) float64 0.1
    wind_speed         (x, y) int32 3
    wind_a             (x, y) int32 1
    wind_b             (x, y) int32 1
    wind_c             (x, y) int32 1
    wind_kh_kw         (x, y) int32 1

In [6]:
tsm_model.get_state_variables()

[Variable(name='water_temp_c', long_name='Water temperature', units='degC', description='TSM state variable for water temperature', use='state', process=CPUDispatcher(<function t_water_c at 0x000001F5AC65DDA0>)),
 Variable(name='surface_area', long_name='Surface area', units='m^2', description='Surface area', use='state', process=<function mock_surface_area at 0x000001F5A949CD60>),
 Variable(name='volume', long_name='Volume', units='m^3', description='Volume', use='state', process=<function mock_volume at 0x000001F5AC65E2A0>)]

In [8]:
print('Variable | Inputs\n------------------')
for i in tsm_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

Variable | Inputs
------------------
air_temp_k | ['air_temp_c']
water_temp_k | ['water_temp_c']
mixing_ratio_air | ['eair_mb', 'pressure_mb']
density_air | ['pressure_mb', 'air_temp_k', 'mixing_ratio_air']
density_water | ['water_temp_c']
esat_mb | ['water_temp_k', 'a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
density_air_sat | ['water_temp_k', 'esat_mb', 'pressure_mb']
ri_number | ['gravity', 'density_air', 'density_air_sat', 'wind_speed']
ri_function | ['ri_number']
lv | ['water_temp_k']
cp_water | ['water_temp_c']
emissivity_air | ['air_temp_k']
wind_function | ['ri_function', 'wind_a', 'wind_b', 'wind_c', 'wind_speed']
q_latent | ['pressure_mb', 'density_water', 'lv', 'wind_function', 'esat_mb', 'eair_mb']
q_sensible | ['wind_kh_kw', 'cp_air', 'density_water', 'wind_function', 'air_temp_k', 'water_temp_k']
q_sediment | ['use_sed_temp', 'pb', 'cps', 'alphas', 'h2', 'sed_temp_c', 'water_temp_c']
dTdt_sediment_c | ['use_sed_temp', 'alphas', 'h2', 'water_temp_c', 'sed_temp_c']
q_longwave_

In [12]:
tsm_model.increment_timestep()
tsm_model.dataset

<xarray.Dataset>
Dimensions:            (year: 2, x: 1, y: 1)
Coordinates:
  * year               (year) int32 0 1
  * x                  (x) float64 1.0
  * y                  (y) float64 1.0
Data variables: (12/52)
    water_temp_c       (year, x, y) float64 20.0 -7.427e+10
    surface_area       (year, x, y) int32 1 1
    volume             (year, x, y) int32 1 1
    use_sed_temp       (x, y) bool True
    stefan_boltzmann   (x, y) int32 10
    cp_air             (x, y) int32 1
    ...                 ...
    q_sediment         (year, x, y) float64 nan -2.572e+03
    dTdt_sediment_c    (year, x, y) float64 nan 0.006001
    q_longwave_down    (year, x, y) float64 nan 5.958e+10
    q_longwave_up      (year, x, y) float64 nan 7.386e+10
    q_net              (year, x, y) float64 nan -3.1e+17
    dTdt_water_c       (year, x, y) float64 nan -7.427e+10

## Next, use the `register_variable` decorator to add a few variables

To do this, make a sub-class of `base.Variable` but with the decorator pointed at the model(s) you want to add the variables too. Note that the `models` argument of the decorator must be either a single sub-class of `base.Model`, or a list of them.

Next, just write instances of the new `base.Variable` sub-class. Each variable's `use` attribute must be set to `static`, `dynamic`, and `state`. Read below about what this means / how you should split up your variables.

Note that anything that needs to be calculated or input into a model should be encapsulated by a variable!

In [11]:
@cwm.base.register_variable(models=CarbonSequestration)
class Variable(cwm.base.Variable):
    ...

In [12]:
Variable(
    name='npp',
    long_name='Net Primary Productivity (NPP)',
    units='g/m^2/year',
    description='The annual average NPP of the forest ecosystem.',
    use='static',
)
Variable(
    name='carbon_content',
    long_name='Carbon Content ratio',
    units='ratio',
    description='The fraction of NPP that is composed of carbon (usually around 50%, but it can vary).',
    use='static',
)

# display the variables we have registered so far
display(CarbonSequestration.get_variable_names())

['npp', 'carbon_content']

### Add our static variables

**Working Definition:** Static variables are any variables that will not change across the course of a simulation, regardless of how many time-steps are run.

Note that one can update static variables if they really want to by re-initializing the model class and providing new static variable inputs.

Here we will use the following static variables:
1. **Net Primary Productivity (NPP)**: The average annual NPP of the forest ecosystem (g/m²/year).
2. **Carbon Content**: The fraction of NPP that is composed of carbon (usually around 50%, but it can vary).

### Add our dynamic variables

**Working Definition:** Dynamic variables are any intermediate variable calculation that don't need to be passed to the next timestep. All dynamic variables need to be associated with a function via the optional `Variable.process` attribute. This "process" function is used to calculate them. **Importantly, the arguments of said function should match the variable names that will be passed in!**

In this simple example we will have only one dynamic variable:
1. **Annual carbon sequestration** (delta_C_annual):

   `delta_C_annual = npp * carbon_content * forest_area`

In [ ]:
@numba.njit
def delta_C_annual(
    npp: float,
    carbon_content: float,
    forest_area: float,
) -> float:
    return npp * carbon_content * forest_area

In [ ]:
Variable(
    name='delta_C_annual',
    long_name='Annual Carbon Delta',
    units='g',
    description='Annual change in forest carbon content',
    use='dynamic',
    process=delta_C_annual,
)

# display the variables we have registered so far
display(CarbonSequestration.get_variable_names())

### Add our state variable

**Working Definition:** A state variable is the main input/output to each timestep. Notably, it can be updated between timesteps to allow interaction with other models. Our model needs to be initialized with state variable values, and no matter what settings are used in initialization, the state variable is stored in our main dataset (keep reading to see this).

Our state variable is the total carbon stock of the forest, which is updated each year:
1. **Total carbon stock** (C_total):

    `C_total = C_total + delta_C_annual`
    
State variables also require a process function.

In [ ]:
@numba.njit
def C_total(
    C_total: float,
    delta_C_annual: float,
) -> float:
    return C_total + delta_C_annual

In [ ]:
Variable(
    name='C_total',
    long_name='Carbon total',
    units='g',
    description='Total forest carbon content',
    use='state',
    process=C_total,
)

Variable(
    name='forest_area',
    long_name='Area of the forest',
    units='m^2',
    description='Area of the forest, may change year by year with deforestation.',
    use='state',
    process=forest_area,
)

# display the variables we have registered so far
display(CarbonSequestration.get_variable_names())

In [ ]:
# for state variables we can see them before initialization
display(CarbonSequestration.get_state_variables())

## Now let's instantiate our new model

To instantiate a model we need to pass in a dictionary with our initial state variable values, any non-default changes to our static variables, and any other optional config settings.

In [ ]:
initial_state_values = {'C_total': 1000, 'forest_area': 1000}
static_variable_values = {
    'carbon_content': 0.5,
    'npp': 10,
}

carbon_model = CarbonSequestration(
    initial_state_values=initial_state_values,  # mandatory
    static_variable_values=static_variable_values,  # mandatory/optional depending on defaults
    track_dynamic_variables=True,  # default is true
    hotstart_dataset=None,  # default is None
    time_dim='year',  # default is "timestep"
)

### All instantiated models have static, dynamic, and state variable properties

In [ ]:
display(carbon_model.state_variables)

In [ ]:
display(carbon_model.static_variables)

In [ ]:
display(carbon_model.dynamic_variables)

### One can access their "computation order" which is calculated using a "dependency tree" approach in `sorter.py`

In [ ]:
carbon_model.computation_order

In [ ]:
print('Variable | Inputs\n------------------')
for i in carbon_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

### Data is stored in `self.dataset`

In [ ]:
carbon_model.dataset

## Running a timestep
All timesteps can be run independently. Optionally, one can update the state values with a float or a `xarray.DataArray`.

In [ ]:
carbon_model.increment_timestep()
carbon_model.dataset

## Running a loop of timesteps

Here we run 100 years of our model with the following hypothetical:
* For the first 50 years deforestation reduces forest area incrementally.
* 50 years in, a program begins that ends deforestation, and the forest grows back incrementally.

**This demonstrates how we can update state variables to interact with other models!**

In [ ]:
%%time
for i in range(100):
    forest_area_change = random.uniform(0.0, 25)
    if i < 50:
        forest_area_change = -forest_area_change
    new_forest_area = (carbon_model.dataset.forest_area + forest_area_change).isel(year=-1)
    carbon_model.increment_timestep(update_state_values={'forest_area': new_forest_area})
carbon_model.dataset

In [ ]:
carbon_model.dataset.hvplot(x='year', y='delta_C_annual', title='delta_C_annual')

In [ ]:
carbon_model.dataset.hvplot(x='year', y='C_total', title='C_total')

# TSM `EnergyBudget` Example

Now that we understand how the code architecture works, we can explore a real example.

In [ ]:
from clearwater_modules.tsm.model import EnergyBudget

In [ ]:
# Confirm that sub-modules are imported
dir(cwm.tsm)

## Start by instantiating a `EnergyBudget`

Initial state variable values are always required. To see the names/info of a model's state variables, we can use `Model.get_state_variables()`.

In [ ]:
EnergyBudget.get_state_variables()

In [ ]:
initial_state_values = {
    'water_temp_c': 1.0,
    'volume': 1.0,
    'surface_area': 1.0,
}

In [ ]:
my_model = EnergyBudget(
    initial_state_values,
    time_dim='my_time_step',
)
my_model

In [ ]:
[i for i in dir(my_model) if i[0] != '_']

## TSM can be initialized with alternative met/temp parameter
**This is an example of a model specific `__init__`**. As of now we are using the defaults.

In [ ]:
my_model.met_parameters

In [ ]:
my_model.temp_parameters

In [ ]:
my_model.time_dim

## All models have static, dynamic, and state variables

In [ ]:
display(my_model.static_variables)

In [ ]:
display(my_model.dynamic_variables)

## One can access their "computation order" which is calculated using a "dependency tree" approach in `sorter.py`

In [ ]:
my_model.computation_order

In [ ]:
for i in my_model.computation_order:
    print(f'{i.name} | {sorter.get_process_args(i.process)}')

## Run 5 timesteps

In [ ]:
TIME_STEPS = 5

In [ ]:
@numba.jit(forceobj=True)
def run_n_timesteps(time_steps: int, model: EnergyBudget):
    for i in range(time_steps):
        model.increment_timestep()

In [ ]:
%%time
run_n_timesteps(TIME_STEPS, my_model)

In [ ]:
my_model.dataset